<a href="https://colab.research.google.com/github/ecsquare/Covid-19/blob/master/Short_text_based_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Topic modeling for short texts is an increasingly impor-tant task due to the prevalence of short texts on the Web.Compared to normal documents, short texts lack of wordfrequency and context information, causing severe sparsityproblems for conventional topic models


In [0]:
!pip install biterm
!pip install pyLDAvis

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk


from string import punctuation
import pickle
from pprint import pprint
import random
import warnings
import time

# numpy, pandas, matplotlib and regular expressions 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# tqdm
from tqdm import tqdm

# spacy
import spacy
from spacy.lang.en import English
import en_core_web_sm

# gensim
import gensim
from gensim import corpora
from gensim.models import CoherenceModel

# nltk
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import words
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# pyLDAvis
import pyLDAvis
import pyLDAvis.gensim


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/vegan_tweets_2020-05-/

/content/gdrive/My Drive/Colab Notebooks/vegan_tweets_2020-05-


In [0]:
ls

 TweetCorpus.txt            vegan_2020-05-03.json   vegan_2020-05-06.json
 vegan_2020-05-01.json      vegan_2020-05-04.json  'vegan_2020-05-07 2.json'
'vegan_2020-05-02 2.json'   vegan_2020-05-05.json


preparing Dataset


In [15]:
import pandas as pd
df1 = pd.read_json (r'vegan_2020-05-01.json',lines=True)
df2 = pd.read_json(r'vegan_2020-05-02 2.json', lines=True)
df3 = pd.read_json(r'vegan_2020-05-03.json', lines=True)
df4 = pd.read_json(r'vegan_2020-05-04.json', lines=True)
df5 = pd.read_json(r'vegan_2020-05-05.json', lines=True)
df6 = pd.read_json(r'vegan_2020-05-06.json', lines=True)
df7 = pd.read_json(r'vegan_2020-05-07 2.json', lines=True)

frames = [df1, df2, df3,df4,df5,df6,df7]
df=pd.concat(frames)
df=df[['id','tweet','hashtags']]
df.columns
df.head(10)

,id,tweet,hashtags
0,1256372793752653825,how bout y’all compare shit that actually make...,[]
1,1256372790497644545,( http://HappyCow.net is an excellent search...,[]
2,1256372760638566401,Whew RT @_underhereye: Brussels Sprouts cooked...,[]
3,1256372759619350528,vegan labouyi (plantiain porridge). one of my ...,[]
4,1256372716141232129,Sounds a bit like a veterinarian vegan sort of...,[]
5,1256372705940705280,"no girl has all three: synesthesia, internal m...",[]
6,1256372679784960006,I definitely need to do my hair in NYC again soon,[]
7,1256372662210879489,how u goin vegan but still beefin w me again?!...,[#newprofilepic]
8,1256372646117117952,Happy Friday! I had some @traderjoes herb pizz...,[#vegan]
9,1256372631130894336,Which do you prefer? Gummies or capsules? ⭐️⭐️...,"[#hairtamin, #vegan, #vegeterian, #hairvitamins]"


Data cleaning

In [0]:
stop_words = ['twitter','the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This','me','my','it','you','your','they','i','u','www','vegan','are','in', 'no','up','s','ve','t','re','i',"was",'were','more','Twitter' ]

In [0]:
def clean_data(text, remove_stop_words=True, stem_words=True):
    # Clean the text, with the option to remove stop_words and to stem words.
   
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"http", " ", text)
    text = re.sub(r"https", " ", text)
    text = re.sub(r"www", " ", text)

    text = re.sub(r".com", " ", text)
    text = re.sub(r".fr", " ", text)
    text = re.sub(r".org", " ", text)
    text = re.sub(r"img", " ", text)


    #remove vegan 
    text = re.sub(r"vegan", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in string.punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
  
    return(text)



In [19]:
df['tweet'] = df['tweet'].apply(lambda x: clean_data(x))
df['hashtags']=  df['hashtags'].apply(str)
df['hashtags']=  df['hashtags'].apply(lambda x: clean_data(x))
df.head(10)

,id,tweet,hashtags
0,1256372793752653825,how bout y all pare shit actual make sens like...,
1,1256372790497644545,happycow net excel search tool vegetarian veg ...,
2,1256372760638566401,whew rt underherey brussel sprout cook vegan b...,
3,1256372759619350528,labouyi plantiain porridg one favorit meal gro...,
4,1256372716141232129,sound bit like veterinarian sort thing,
5,1256372705940705280,girl has all three synesthesia intern monologu,
6,1256372679784960006,i definit need do hair nyc again soon,
7,1256372662210879489,how goin still beefin w again newprofilep pic ...,newprofilep
8,1256372646117117952,happi friday i had some traderjo herb pizza do...,
9,1256372631130894336,do prefer gummi capsul hairtamin vegeterian ha...,hairtamin vegeterian hairvitamin


In [20]:
df["Tweet Length"]= df["tweet"].str.len() 
df.head(10)

,id,tweet,hashtags,Tweet Length
0,1256372793752653825,how bout y all pare shit actual make sens like...,,86
1,1256372790497644545,happycow net excel search tool vegetarian veg ...,,168
2,1256372760638566401,whew rt underherey brussel sprout cook vegan b...,,87
3,1256372759619350528,labouyi plantiain porridg one favorit meal gro...,,77
4,1256372716141232129,sound bit like veterinarian sort thing,,38
5,1256372705940705280,girl has all three synesthesia intern monologu,,46
6,1256372679784960006,i definit need do hair nyc again soon,,37
7,1256372662210879489,how goin still beefin w again newprofilep pic ...,newprofilep,56
8,1256372646117117952,happi friday i had some traderjo herb pizza do...,,230
9,1256372631130894336,do prefer gummi capsul hairtamin vegeterian ha...,hairtamin vegeterian hairvitamin,70


In [0]:
df = df[df['Tweet Length'] >10]

In [0]:
import matplotlib.pyplot as plt

df['Tweet Length'].plot(kind='bar');

In [46]:
print("mean=", df['Tweet Length'].mean(), "max=", df['Tweet Length'].max(), "min=",df['Tweet Length'].min())

mean= 98.13692187091716 max= 451 min= 16


In [0]:
df.hashtags.value_counts()


In [23]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/Topic_modeling

/content/gdrive/My Drive/Colab Notebooks/Topic_modeling


**BiTerM** 

In [47]:
data = df['tweet'].values.tolist()
len(data)

11481

In [0]:
with open('TweetCorpus.txt', 'w') as f:
    for i in data:
        f.write("%s\n" % i)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(data).toarray()

In [0]:
import numpy as np 
from biterm.utility import vec_to_biterms

vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X[:50])

In [0]:
biterms

In [0]:
from biterm.btm import oBTM

btm = oBTM(num_topics=20, V=vocab)

In [0]:
topics = btm.fit_transform(biterms, iterations=20)

In [0]:
topics

In [0]:
# Enabling pyLDAvis
pyLDAvis.enable_notebook()

In [0]:
import pyLDAvis
print("\n\n Visualize Topics ..")

vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X[:50], axis=1), vocab, np.sum(X[:50], axis=0))
pyLDAvis.save_html(vis, 'testoutput_btm.html')
vis



 Visualize Topics ..


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.137116 -0.081291       1        1  13.144218
9      0.325561  0.168392       2        1  13.008013
6      0.096407 -0.104202       3        1  12.392889
8     -0.129110  0.089566       4        1  11.764054
1     -0.113946  0.067857       5        1  11.415697
5      0.093013  0.054820       6        1   9.582344
7     -0.015487 -0.252321       7        1   8.936115
4      0.111915 -0.131710       8        1   8.499492
0     -0.063860  0.006980       9        1   5.981698
3     -0.167377  0.181908      10        1   5.275481, topic_info=          Term      Freq      Total Category  logprob  loglift
20442     with  5.000000   5.000000  Default  30.0000  30.0000
14636      pic  9.000000   9.000000  Default  29.0000  29.0000
3299       are  4.000000   4.000000  Default  28.0000  28.0000
15750       re  2.000000   2.000000  Default  27.0000  27.0000
7115       eat  4.000000   4.000000  Default  26.0000  26.0000
...        ...       ...        ...      ...      ...      ...
5763      cook  1.011212   3.504133  Topic10  -3.5993   1.6993
8256      food  0.948031   4.404220  Topic10  -3.6638   1.4062
19440       up  0.916441   5.212696  Topic10  -3.6977   1.2037
14636      pic  1.200756   9.356184  Topic10  -3.4275   0.8890
19150  twitter  1.169165  10.574261  Topic10  -3.4541   0.7400

[447 rows x 6 columns], token_table=       Topic      Freq                 Term
term                                       
230        8  1.174378                  120
267        2  0.817475  1255813063925776384
859        4  0.760408        185bt92gvuurv
1162       2  0.817475                 2015
1419       3  0.961698              2vs8gpu
...      ...       ...                  ...
20557      6  0.700819                would
20746      3  0.961698           y3fdjcgswr
20857     10  1.049837               yellow
20980      4  1.044128               yogurt
21216      7  1.344327                  zoe

[456 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 7, 9, 2, 6, 8, 5, 1, 4])

Methode 2: **GSDMM**

In [0]:

import random

class GibbsSamplingDMM(object):
	
	numDocuments = 0
	numWordsInCorpus = 0
	word2IdVocabulary = {}
	id2WordVocabulary = {}
	documents = []
	occurenceToIndexCount = []
	topicAssignments = []
	docTopicCount = []
	topicWordCount = []
	sumTopicWordCount = []
	multiPros = []
	betaSum=0.

	def __init__(self, paramters):
		super(GibbsSamplingDMM, self).__init__()
		self.corpus = paramters["corpus"]
		self.output = paramters["output"]
		self.ntopics = int(paramters["ntopics"])
		self.alpha = float(paramters["alpha"])
		self.beta = float(paramters["beta"])
		self.niters = int(paramters["niters"])
		self.twords = int(paramters["twords"])
		self.name = paramters["name"]

	def analyseCorpus(self):
		indexWord=0
		data = open(self.corpus).read().splitlines()[:50]
		for doc in data:
			document = []
			wordOccurenceToIndexInDocCount = {}
			wordOccurenceToIndexInDoc = []
			if doc.rstrip!=None:
				words = doc.rstrip().split()
				for word in words:

					if word in self.word2IdVocabulary:
						document.append(self.word2IdVocabulary[word])
					else:
						self.word2IdVocabulary[word]=indexWord
						self.id2WordVocabulary[indexWord]=word
						document.append(indexWord)
						indexWord+=1

					if word in wordOccurenceToIndexInDocCount:
						wordOccurenceToIndexInDocCount[word]+=1
					else:
						wordOccurenceToIndexInDocCount[word]=1

					wordOccurenceToIndexInDoc.append(wordOccurenceToIndexInDocCount[word])

				self.numWordsInCorpus+=len(document)
				self.numDocuments+=1
				self.documents.append(document)
				self.occurenceToIndexCount.append(wordOccurenceToIndexInDoc)

		self.betaSum=len(self.word2IdVocabulary)*self.beta


	def topicAssigmentInitialise(self):
		self.docTopicCount = [0 for x in range(self.ntopics)]
		self.sumTopicWordCount = [0 for x in range(self.ntopics)]

		for i in range(self.ntopics):
				self.topicWordCount.append([0 for x in range(len(self.word2IdVocabulary))])

		for i in range (self.numDocuments):
			topic = random.randint(0,self.ntopics-1)
			self.docTopicCount[topic]+=1
			
			for j in range (len(self.documents[i])):
				self.topicWordCount[topic][self.documents[i][j]]+=1
				self.sumTopicWordCount[topic]+=1

			self.topicAssignments.append(topic)

	def nextDiscrete(self,a):
		b = 0.

		for i in range(len(a)):
			b+=a[i]

		r = random.uniform(0.,1.)*b
		
		b=0.
		for i in range (len(a)):
			b+=a[i]
			if(b>r):
				return i
		return len(a)-1

	def sampleInSingleIteration(self,x):
		print ("iteration: "+str(x))
		for d in range(self.numDocuments):
			topic = self.topicAssignments[d]
			self.docTopicCount[topic]-=1
			docSize = len(self.documents[d])
			document = self.documents[d]

			for w in range(docSize):
				word = document[w]
				self.topicWordCount[topic][word]-=1
				self.sumTopicWordCount[topic]-=1
			

			for t in range(self.ntopics):
				self.multiPros[t] = self.docTopicCount[t]+self.alpha

				for w in range(docSize):
					word = document[w]
					self.multiPros[t] *= (self.topicWordCount[t][word]+self.beta+self.occurenceToIndexCount[d][w]-1)/(self.sumTopicWordCount[t]+w+self.betaSum)
				
					
			#print self.multiPros
			topic = self.nextDiscrete(self.multiPros)
			#print topic

			self.docTopicCount[topic]+=1

			for w in range(docSize):
				word = document[w]
				self.topicWordCount[topic][word]+=1
				self.sumTopicWordCount[topic]+=1

			self.topicAssignments[d] = topic

	def inference(self):
		self.multiPros = [0 for x in range(self.ntopics)]
		[self.sampleInSingleIteration(x) for x in range(self.niters)]

	def writeTopicAssignments(self):
		file = open(self.output+self.name+"topicAssignments.txt","w")
		#for i in range(self.numDocuments):
		[file.write(str(self.topicAssignments[i])+"\n") for i in range(self.numDocuments)]



	def writeTopTopicalWords(self):
		file = open(self.output+self.name+"topWords.txt","w") 
		for t in range(self.ntopics):
			wordCount = {w:self.topicWordCount[t][w] for w in range(len(self.word2IdVocabulary))}
			
			count =0
			string="Topic "+str(t)+": "
			
			for index in sorted(wordCount, key=wordCount.get, reverse=True):
				string += self.id2WordVocabulary[index]+" "
				count+=1
				if count>=self.twords:
					file.write(string+"\n") 
					# print string
					break
		file.close()

In [0]:
params = {"corpus":"TweetCorpus.txt", 
          "output": "outGSDMM", 
          "ntopics": "20", 
          "alpha":"0.1",
          "beta":"0.1", 
          "niters":"300",
          "twords":"20",
          "name":""}

model = GibbsSamplingDMM(params)
model.analyseCorpus()
model.topicAssigmentInitialise()
model.inference()
	
print ("Writing Results")
model.writeTopTopicalWords()
model.writeTopicAssignments()

Method 3: **LDA ** 


Firstly, I am creating a dictionary from our data, then I am converting it into a bag-of-words corpus and then saving the dictionary and corpus for future use using pickle library

In [0]:

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [0]:
text_data = []
for row in tqdm(data):
    tokens = tokenize(row)
    text_data.append(tokens)

In [0]:
print("Generating corpus and dictionary...")

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [0]:
%%time
print("Training LDA model on train dataframe...")

NUM_TOPICS = 20
start_time = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=NUM_TOPICS, 
                                           random_state=89, 
                                           update_every=1,  
                                           id2word=dictionary, 
                                           passes=42, 
                                           alpha='auto', 
                                           per_word_topics=True)

ldamodel.save('model' + str(NUM_TOPICS) + '.gensim')
train_time = time.time() - start_time
print("Training Time --- %s seconds " % (round(train_time, 2)))

In [0]:
print("Allocated models of each topic...")

topics_train = []
topics = ldamodel.print_topics()
for topic in topics:
    pprint(topic)
    topics_train.append(topic)
    

with open("results_train.txt", 'w') as f:
    pprint(topics_train, stream=f)

In [65]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=text_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.411081838512219


In [64]:
# Visualizing our optimal model with 100 topics
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(vis, 'testoutput_LDA.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.418643  0.109221       1        1  47.792397
11     0.021215  0.196596       2        1   4.644135
12    -0.102038 -0.047738       3        1   4.122420
0     -0.042175  0.234737       4        1   3.705200
2     -0.082375 -0.173509       5        1   3.266760
15    -0.049207 -0.165863       6        1   3.186800
7     -0.005643 -0.067776       7        1   3.148028
13     0.049484 -0.028492       8        1   3.048759
14    -0.014630 -0.054250       9        1   2.915936
10    -0.028720 -0.111972      10        1   2.760430
9      0.091276  0.132873      11        1   2.543028
18     0.074138 -0.017414      12        1   2.478353
3      0.053805  0.039742      13        1   2.423259
19     0.031978 -0.036392      14        1   2.420982
1      0.123763  0.089583      15        1   2.302796
5      0.009368 -0.046584      16        1   2.045928
16     0.074978  0.002662      17        1   2.036738
8      0.025609 -0.052587      18        1   1.768991
6      0.087280 -0.008769      19        1   1.712704
4      0.100536  0.005933      20        1   1.676351, topic_info=         Term         Freq        Total Category  logprob  loglift
78          i  6501.000000  6501.000000  Default  30.0000  30.0000
50      vegan  2352.000000  2352.000000  Default  29.0000  29.0000
143         b   619.000000   619.000000  Default  28.0000  28.0000
45        pic  1946.000000  1946.000000  Default  27.0000  27.0000
115      with  1729.000000  1729.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
173   chicken    40.252106   261.350067  Topic20  -4.4314   2.2179
2496   season    21.885014    47.876656  Topic20  -5.0408   3.3057
383     pleas    29.192953   177.122162  Topic20  -4.7527   2.2856
485     video    20.235079   114.597008  Topic20  -5.1192   2.3545
45        pic    19.274086  1946.453369  Topic20  -5.1678  -0.5265

[1047 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
3185      5  0.949751      00
5398     14  0.971541     000
1479      3  0.054167      04
1479      6  0.433334      04
1479     13  0.487501      04
...     ...       ...     ...
561       6  0.665942  youtub
561      17  0.295038  youtub
561      20  0.033719  youtub
925       7  0.926606   yummi
925      20  0.057913   yummi

[1633 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 12, 13, 1, 3, 16, 8, 14, 15, 11, 10, 19, 4, 20, 2, 6, 17, 9, 7, 5])